# Binary Classification Model for Disease Prediction


# Part 1: Installation of necessary libraries

In [34]:
import numpy as np
import pandas as pd
import os 
import random
import tensorflow
from scipy.spatial import distance


# Part 2: Download of patient data with corresponding disease status

In [35]:

disease_data = pd.read_csv('Training.csv')



# Part 3 : Exploratory Data Analysis


In [36]:
disease_data_description = disease_data.describe()
print(disease_data_description)
#disease_data.dtypes

               AB            AF           AH          AM          AR   
count  617.000000    617.000000   617.000000  617.000000  617.000000  \
mean     0.477149   3502.013221   118.624513   38.968552   10.128242   
std      0.468388   2300.322717   127.838950   69.728226   10.518877   
min      0.081187    192.593280    85.200147    3.177522    8.138688   
25%      0.252107   2197.345480    85.200147   12.270314    8.138688   
50%      0.354659   3120.318960    85.200147   20.533110    8.138688   
75%      0.559763   4361.637390   113.739540   39.139886    8.138688   
max      6.161666  28688.187660  1910.123198  630.518230  178.943634   

               AX          AY          AZ           BC           BD   ...   
count  617.000000  617.000000  617.000000   617.000000    617.000000  ...  \
mean     5.545576    0.060320   10.566447     8.053012   5350.388655  ...   
std      2.551696    0.416817    4.350645    65.166943   3021.326641  ...   
min      0.699861    0.025578    3.396778  

In [37]:

#print(disease_data.isnull().sum())
disease_data=disease_data.dropna()
print(disease_data.isnull().sum())
disease_data['EJ'] = disease_data['EJ'].map({'A': 0, 'B': 1})

Id       0
AB       0
AF       0
AH       0
AM       0
AR       0
AX       0
AY       0
AZ       0
BC       0
BD       0
BN       0
BP       0
BQ       0
BR       0
BZ       0
CB       0
CC       0
CD       0
CF       0
CH       0
CL       0
CR       0
CS       0
CU       0
CW       0
DA       0
DE       0
DF       0
DH       0
DI       0
DL       0
DN       0
DU       0
DV       0
DY       0
EB       0
EE       0
EG       0
EH       0
EJ       0
EL       0
EP       0
EU       0
FC       0
FD       0
FE       0
FI       0
FL       0
FR       0
FS       0
GB       0
GE       0
GF       0
GH       0
GI       0
GL       0
Class    0
dtype: int64


# Part 4 : Generating New Data Features

In [38]:
def euclidean_distance(sample1, sample2):
    """Calculate the Euclidean distance between two samples."""
    return distance.euclidean(sample1, sample2)


# Part 5 : Creation of a machine learning model

# Part 6 : Analysis of models efficiency and loss values

# Part 7 : Log of models